In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator

load_dotenv()

model= ChatOpenAI(model='gpt-4o-mini')

In [ ]:
class EvaluationSchema(BaseModel):
    #If we simply try to give these in the form of prompt to the model then it might make mistakes i.e. the changes of the model giving different outputs (like for score it might give 'six' instead of '6' as the score). To avoid this we used the concept of structured outputs
    feedback: str= Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [ ]:
structured_model=model.with_structured_output(EvaluationSchema)

In [ ]:
class UPSCState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add] #as we need to store individual scores(in form of list) from each of language, analysis and clarity node so we use 'operator.add' function that adds all the lists into 1 single list
    avg_score: float

In [ ]:
essay='''Membrane proteins are essential components of biological membranes, facilitating communication and
exchange between the cell and its environment. They can be classified into integral (intrinsic) membrane
proteins, which are embedded within or span the lipid bilayer, and peripheral (extrinsic) membrane proteins,
which are loosely associated with the membrane surface. These proteins are involved in diverse functions such
as signaling, transport, and structural support, contributing critically to cellular homeostasis and adaptability.'''

In [ ]:
def evaluate_language(state: UPSCState):
    prompt=f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']} '
    output=structured_model.invoke(prompt)
    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}


In [ ]:
def evaluate_analysis(state: UPSCState):
    prompt=f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']} '
    output=structured_model.invoke(prompt)
    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [ ]:
def evaluate_thought(state: UPSCState):
    prompt=f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']} '
    output=structured_model.invoke(prompt)
    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [ ]:
def final_evaluation(state: UPSCState):

    #summary feedback
    prompt= f'Based on the following feedbacks create a summarized feedback \n language feedback - {state['language_feedback']} \n depth of analysis feedback - {state['analysis_feedback']} \n, clarity of thought - {state['clarity_feedback']}'

    overall_feedback=model.invoke(prompt).content

    #avg calculation
    avg_score=sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [ ]:
#create the graph
graph= StateGraph(UPSCState)

#add nodes
graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

#add edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')
#parallel
graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')
graph.add_edge('final_evaluation',END)

#compile
workflow=graph.compile()

In [ ]:
#to see the workflow
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
#execute

initial_state={'essay': essay}

final_state=workflow.invoke(initial_state)

print(final_state)
